In [1]:
import caffe
import os
from pylab import *
import sys
from PIL import Image
import numpy as np
import cv2
import torch
from torch.autograd import Variable
import torchvision

%matplotlib inline

In [2]:
mean_filename='SpotGarbage_GarbNet/garbnet_mean.binaryproto'
deploy_filename = 'SpotGarbage_GarbNet/deploy_garbnet.prototxt'
caffemodel_file = 'SpotGarbage_GarbNet/garbnet_fcn.caffemodel'

proto_data = open(mean_filename, "rb").read()
a = caffe.io.caffe_pb2.BlobProto.FromString(proto_data)
mean  = caffe.io.blobproto_to_array(a)[0]

net = caffe.Net(deploy_filename,caffemodel_file,caffe.TEST)
for item,blob in net.params.items():
    print(item,blob[0].data.shape)
    
print('\n')
for item,blob in net.blobs.items():
    print(item,blob.data.shape)

conv1 (96, 3, 11, 11)
conv2 (256, 48, 5, 5)
conv3 (384, 256, 3, 3)
conv4 (384, 192, 3, 3)
conv5 (256, 192, 3, 3)
fc6_gb_conv (512, 256, 6, 6)
fc7_gb_conv (256, 512, 1, 1)
fc8_gb_conv (2, 256, 1, 1)


data (1, 3, 227, 227)
conv1 (1, 96, 55, 55)
pool1 (1, 96, 27, 27)
conv2 (1, 256, 27, 27)
pool2 (1, 256, 13, 13)
conv3 (1, 384, 13, 13)
conv4 (1, 384, 13, 13)
conv5 (1, 256, 13, 13)
pool5 (1, 256, 6, 6)
fc6_gb_conv (1, 512, 1, 1)
fc7_gb_conv (1, 256, 1, 1)
fc8_gb_conv (1, 2, 1, 1)
prob (1, 2, 1, 1)


In [14]:
import torch
import imp
MainModel = imp.load_source('MainModel', "SpotGarbage_GarbNet/pytorch_garbnet.py")

the_model = torch.load("SpotGarbage_GarbNet/pytorch_garbnet.pth")
the_model.eval()

KitModel(
  (conv1): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4))
  (conv2): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), groups=2)
  (conv3): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), groups=2)
  (conv5): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), groups=2)
  (fc6_gb_conv): Conv2d(256, 512, kernel_size=(6, 6), stride=(2, 2))
  (fc7_gb_conv): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
  (fc8_gb_conv): Conv2d(256, 2, kernel_size=(1, 1), stride=(1, 1))
)

In [4]:


def load_dataset(path):
    #1.resize for the network 277x277 (as garbdemo instructed)
    #2.convert to tensor (gpu array)
    #3.will normalize data between -1 and 1
    transform = torchvision.transforms.Compose(
                    [torchvision.transforms.Resize((277,277)),
                    torchvision.transforms.ToTensor(),
                     torchvision.transforms.Normalize(
                         (108.39786852, 115.89358715, 119.99467375), (1.0, 1.0, 1.0)
                     )
                    ])
    
    #to import images from the disk
    dataset = torchvision.datasets.ImageFolder(
        root=path,
        transform=transform
    )
    
    #creates an iterable for the dataset to use in a forloop
    dataset_loader = torch.utils.data.DataLoader(
        dataset,
        num_workers=2,
        shuffle=False
    )
    return dataset_loader

'''
image directory must be in the same location of this script
images/garbage/img0.jpg,img1.jpg,img2.jpg
images/notgarbage/img3.jpg,img4.jpg

pytorch will label the classes based on the number of folders there are.
garbage is folder 0 so class label is 0
notgarbage is folder 1 so class label is 1

only need the directory where all class folders are for argument
'''
dataset = load_dataset('images/')

In [5]:
print('Labels: \n 0.Garbage \n 1.Not Garbage\n')
for data, target in dataset:
    #prints [batchsize, channel size (RGB->3), height, width], label 
    print('Tensor (image to array) size: ', data.size(), 'Target label: ', target,end='\n\n')

Labels: 
 0.Garbage 
 1.Not Garbage

Tensor (image to array) size:  torch.Size([1, 3, 277, 277]) Target label:  tensor([0])

Tensor (image to array) size:  torch.Size([1, 3, 277, 277]) Target label:  tensor([0])

Tensor (image to array) size:  torch.Size([1, 3, 277, 277]) Target label:  tensor([0])

Tensor (image to array) size:  torch.Size([1, 3, 277, 277]) Target label:  tensor([1])

Tensor (image to array) size:  torch.Size([1, 3, 277, 277]) Target label:  tensor([1])



In [6]:
import pdb

for data, target in dataset:
    pred = the_model(data)
    print(pred)
    print(pred.argmax(), target)

tensor([[[[1.0000e+00, 1.0000e+00],
          [1.0000e+00, 1.0000e+00]],

         [[2.0495e-08, 1.3384e-08],
          [9.6618e-09, 7.9385e-09]]]], grad_fn=<SoftmaxBackward>)
tensor(3) tensor([0])
tensor([[[[1.0000e+00, 1.0000e+00],
          [1.0000e+00, 1.0000e+00]],

         [[2.1155e-08, 1.5461e-08],
          [1.0189e-08, 8.2807e-09]]]], grad_fn=<SoftmaxBackward>)
tensor(3) tensor([0])
tensor([[[[1.0000e+00, 1.0000e+00],
          [1.0000e+00, 1.0000e+00]],

         [[2.3340e-08, 1.6308e-08],
          [1.1409e-08, 9.6301e-09]]]], grad_fn=<SoftmaxBackward>)
tensor(3) tensor([0])
tensor([[[[1.0000e+00, 1.0000e+00],
          [1.0000e+00, 1.0000e+00]],

         [[2.8744e-08, 1.9176e-08],
          [1.4363e-08, 1.1266e-08]]]], grad_fn=<SoftmaxBackward>)
tensor(3) tensor([1])
tensor([[[[1.0000e+00, 1.0000e+00],
          [1.0000e+00, 1.0000e+00]],

         [[2.8395e-08, 1.9204e-08],
          [1.5104e-08, 1.1878e-08]]]], grad_fn=<SoftmaxBackward>)
tensor(3) tensor([1])


SpotGarbage_GarbNet/pytorch_garbnet.py:67: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  prob            = F.softmax(fc8_gb_conv)


In [7]:
def gatherImages(folder,imageNames=None):
    images = []
    names = []
    files = os.listdir(folder)
    total = len(files)
    #print('Total %d images in folder %s' % (total,folder))
    for i in os.listdir(folder):
        try:
            if imageNames is None or i in imageNames:
                example_image = folder+'/'+i
                input_image = Image.open(example_image)
                images.append(input_image)
                names.append(i)
        except:
            pass

    return images,names


In [8]:


def resizeForFCN(image,size):
    w,h = image.size
    if w<h:
        return image.resize((int(227*size),int((227*h*size)/w))) #227x227 is input for regular CNN
    else:
        return image.resize((int((227*w*size)/h),int(227*size)))
    
def getSegmentedImage(test_image, probMap,thresh):
    kernel = np.ones((6,6),np.uint8)
    wt,ht = test_image.size
    out_bn = np.zeros((ht,wt),dtype=uint8)
    res = 0
    for h in range(probMap.shape[0]):
        
        for k in range(probMap.shape[1]):
            if probMap[h,k] > thresh:
                res += 1
                x1 = h*62 #stride 2 at fc6_gb_conv equivalent to 62 pixels stride in input
                y1 = k*62
                for hoff in range(x1,227+x1):
                    if hoff < out_bn.shape[0]:
                        for koff in range(y1,227+y1):
                            if koff < out_bn.shape[1]:
                                out_bn[hoff,koff] = 255
    print("count ",res)
    edge = cv2.Canny(out_bn,200,250)
    box = cv2.dilate(edge,kernel,iterations = 3)
    
    or_im_ar = np.array(test_image)
    or_im_ar[:,:,1] = (or_im_ar[:,:,1] | box)
    or_im_ar[:,:,2] = or_im_ar[:,:,2] * box + or_im_ar[:,:,2]
    or_im_ar[:,:,0] = or_im_ar[:,:,0] * box + or_im_ar[:,:,0]
    
    return Image.fromarray(or_im_ar)
    
    
def getPredictionsFor(images,names,size,thresh,output_folder):
    for i in range(len(images)):
#         try:
            test_image = resizeForFCN(images[i],size)
            
            in_ = np.array(test_image,dtype = np.float32)
            in_ = in_[:,:,::-1]
            print('min before scaling',np.amin(in_))
            print('max before scaling',np.amax(in_))
            in_ -= np.array(mean.mean(1).mean(1))
            print('mean scaling', np.array(mean.mean(1).mean(1)))
            print('min after scaling',np.amin(in_))
            print('max after scaling',np.amax(in_))
            in_ = in_.transpose((2,0,1))
            if i ==0:
                print(in_)
            #print('in_',in_)
            net.blobs['data'].reshape(1,*in_.shape)
            net.blobs['data'].data[...] = in_
            net.forward()
            #print(net.blobs['prob'].data[0,1])
            probMap =net.blobs['prob'].data[0,1]
            print('probmap: ',probMap.shape)
            print( names[i]+'...',)
            if len(np.where(probMap>thresh)[0]) > 0:
                print( 'Garbage!')
            else:
                print('Not Garbage!')
            
            out_ = getSegmentedImage(test_image, probMap,thresh)
            out_.save(output_folder + '/output_' + names[i])
#         except:
#             pass

In [9]:
#specify 'input' folder containing images for prediction
images,names = gatherImages('images/garbage')
print(images[0])
#images[0].show()
#specify 'output' folder to store segmented predictions
getPredictionsFor(images,names,4,0.999,'output')

<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=550x367 at 0x7F7957A14510>
min before scaling 0.0
max before scaling 255.0
mean scaling [108.39786852 115.89358715 119.9946737 ]
min after scaling -119.994675
max after scaling 146.60213
[[[-31.39787  -31.39787  -67.397865 ... -38.39787  -37.39787  -37.39787 ]
  [-31.39787  -31.39787  -67.397865 ... -38.39787  -37.39787  -37.39787 ]
  [-28.39787  -28.39787  -72.397865 ... -38.39787  -36.39787  -36.39787 ]
  ...
  [ 40.60213   40.60213   28.60213  ...  61.60213   44.60213   44.60213 ]
  [ 66.602135  66.602135  44.60213  ...  57.60213   47.60213   47.60213 ]
  [ 66.602135  66.602135  44.60213  ...  57.60213   47.60213   47.60213 ]]

 [[-33.89359  -33.89359  -69.893585 ... -33.89359  -32.89359  -32.89359 ]
  [-33.89359  -33.89359  -69.893585 ... -33.89359  -32.89359  -32.89359 ]
  [-30.893587 -30.893587 -75.893585 ... -33.89359  -31.893587 -31.893587]
  ...
  [ 43.10641   43.10641   31.106413 ...  55.10641   38.10641   38.10641 ]
  [ 

In [10]:
imshow(Image.open('input/sample_2_garbage.jpg'))
figure()
imshow(Image.open('output/output_sample_2_garbage.jpg'))

FileNotFoundError: [Errno 2] No such file or directory: 'input/sample_2_garbage.jpg'